In [1]:
! pip install faker==23.0.0

In [2]:
# VARIANT Data Type Exploration in Databricks
# This notebook demonstrates various features and capabilities of the VARIANT data type

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json
import random
from datetime import datetime, timedelta
from faker import Faker
import os

# =============================================================================
# CONFIGURATION PARAMETERS
# =============================================================================
# Update these parameters to customize the notebook for your environment
CATALOG = "pavan_naidu"                    # Unity Catalog name
SCHEMA = "json"                            # Schema name within the catalog
VOLUME = "raw_data"                        # Volume name for storing data
# =============================================================================

def get_spark() -> SparkSession:
    try:
        from databricks.connect import DatabricksSession
        return DatabricksSession.builder.getOrCreate()
    except Exception:
        return SparkSession.builder.getOrCreate()

spark = get_spark()
fake = Faker()



# Set catalog and schema using parameters
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {SCHEMA}")

print("✅ Spark session initialized")
print(f"Current catalog: {spark.sql('SELECT current_catalog()').collect()[0][0]}")
print(f"Current schema: {spark.sql('SELECT current_schema()').collect()[0][0]}")
print(f"Configuration: CATALOG={CATALOG}, SCHEMA={SCHEMA}, VOLUME={VOLUME}")

✅ Spark session initialized


Current catalog: pavan_naidu


Current schema: json
Configuration: CATALOG=pavan_naidu, SCHEMA=json, VOLUME=raw_data


## 1. Setup: Create UC Volume and Generate Fake JSON Data

First, let's create a folder in Unity Catalog volume and generate diverse JSON data to showcase VARIANT functionality.


In [3]:
# Create or verify volume exists
try:
    spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.{SCHEMA}.{VOLUME}")
    print(f"✅ Volume '{VOLUME}' is ready")
except Exception as e:
    print(f"Volume might already exist or error: {e}")

# Get the volume path
volume_path = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}"
print(f"Volume path: {volume_path}")


✅ Volume 'raw_data' is ready
Volume path: /Volumes/pavan_naidu/json/raw_data


In [4]:
# Generate fake user data to demonstrate VARIANT capabilities

def generate_user_data(num_records=1000):
    """Generate user data with varying structures to simulate schema evolution"""
    users = []
    
    for i in range(num_records):
        # Basic user structure (all users have these fields)
        user = {
            "user_id": str(fake.uuid4()),
            "username": fake.user_name(),
            "email": fake.email(),
            "name": fake.name(),
            "age": random.randint(18, 80),
            "created_at": str(fake.date_time_between(start_date='-5y', end_date='now')),
            "last_login": str(fake.date_time_between(start_date='-30d', end_date='now'))
        }
        
        # Profile information (nested structure)
        user["profile"] = {
            "bio": fake.text(max_nb_chars=200),
            "occupation": fake.job(),
            "company": fake.company(),
            "interests": [fake.word() for _ in range(random.randint(1, 5))],
            "skills": [fake.job() for _ in range(random.randint(0, 3))]
        }
        
        # Address information (nested with multiple levels)
        user["address"] = {
            "street": fake.street_address(),
            "city": fake.city(),
            "state": fake.state(),
            "country": fake.country(),
            "postal_code": fake.postcode(),
            "coordinates": {
                "latitude": float(fake.latitude()),
                "longitude": float(fake.longitude())
            }
        }
        
        # Preferences (nested boolean flags)
        user["preferences"] = {
            "newsletter": bool(random.choice([True, False])),
            "notifications": {
                "email": bool(random.choice([True, False])),
                "sms": bool(random.choice([True, False])),
                "push": bool(random.choice([True, False])),
                "frequency": random.choice(["daily", "weekly", "monthly", "never"])
            },
            "privacy": {
                "profile_visible": bool(random.choice([True, False])),
                "show_email": bool(random.choice([True, False])),
                "show_location": bool(random.choice([True, False]))
            }
        }
        
        # Simulate schema evolution - newer users have additional fields
        if i > num_records * 0.3:  # 70% of users have these newer fields
            user["social_media"] = {
                "twitter": f"@{fake.user_name()}" if random.random() > 0.3 else None,
                "linkedin": fake.url() if random.random() > 0.5 else None,
                "github": f"github.com/{fake.user_name()}" if random.random() > 0.7 else None
            }
            
        if i > num_records * 0.6:  # 40% of users have these newest fields
            user["subscription"] = {
                "tier": random.choice(["free", "basic", "premium", "enterprise"]),
                "start_date": str(fake.date_between(start_date='-2y', end_date='today')),
                "auto_renew": bool(random.choice([True, False]))
            }
            user["metrics"] = {
                "login_count": random.randint(1, 1000),
                "posts_created": random.randint(0, 500),
                "comments_made": random.randint(0, 2000),
                "last_activity": str(fake.date_time_between(start_date='-7d', end_date='now'))
            }
        
        # Some users have optional fields (simulate sparse data)
        if random.random() > 0.7:
            user["phone"] = fake.phone_number()
        
        if random.random() > 0.8:
            user["referral_code"] = fake.bothify(text='REF-####-????')
            user["referred_by"] = str(fake.uuid4()) if random.random() > 0.5 else None
        
        users.append(user)
    
    return users

# Generate sample user data
print("Generating fake user data...")
user_data = generate_user_data(1000)
print(f"✅ Generated {len(user_data)} user records with evolving schemas")

# Show sample records demonstrating schema evolution
print("\nSample user records showing schema evolution:")
print("\n1. Basic user (early schema):")
print(json.dumps(user_data[100], indent=2)[:600] + "...")
print("\n2. User with social media (mid evolution):")
print(json.dumps(user_data[500], indent=2)[:600] + "...")
print("\n3. User with full schema (latest):")
print(json.dumps(user_data[900], indent=2)[:600] + "...")


Generating fake user data...
✅ Generated 1000 user records with evolving schemas

Sample user records showing schema evolution:

1. Basic user (early schema):
{
  "user_id": "1740eff5-bd8f-42a1-9bdf-3621cdf13b15",
  "username": "troymiller",
  "email": "jeffrey60@example.com",
  "name": "Michael Chen",
  "age": 78,
  "created_at": "2022-11-04 15:16:08.258875",
  "last_login": "2025-09-18 01:36:04.311718",
  "profile": {
    "bio": "Answer process school. Just clear certainly station down. Protect bit maybe or.\nKid drop letter beat. Human listen woman door whom beyond.",
    "occupation": "Designer, jewellery",
    "company": "Oneill Ltd",
    "interests": [
      "ok",
      "front",
      "magazine"
    ],
    "skills": [
      "Furniture designer...

2. User with social media (mid evolution):
{
  "user_id": "d350e4d7-b2f2-43a6-bbf1-81b29d0b9b45",
  "username": "hamiltonanna",
  "email": "lori33@example.net",
  "name": "Christopher Scott",
  "age": 66,
  "created_at": "2022-07-25 13:

In [5]:
# Create a users folder in the volume
users_folder = f"{volume_path}/users"
try:
    dbutils.fs.mkdirs(users_folder)
    print(f"✅ Created folder: {users_folder}")
except Exception as e:
    print(f"Folder might already exist: {e}")

# Save JSON data to users folder
json_file_path = f"{users_folder}/user_data.json"

# Write JSON data to file
dbutils.fs.put(json_file_path, json.dumps(user_data), overwrite=True)
print(f"✅ Saved JSON data to: {json_file_path}")

# Also save as individual JSON lines for easier processing
jsonl_file_path = f"{users_folder}/user_data.jsonl"
jsonl_content = "\n".join([json.dumps(user) for user in user_data])
dbutils.fs.put(jsonl_file_path, jsonl_content, overwrite=True)
print(f"✅ Saved JSONL data to: {jsonl_file_path}")

# # Optionally, save data partitioned by subscription tier for those who have it
# print("\n📁 Creating partitioned data by subscription tier...")
# for user in user_data:
#     if "subscription" in user:
#         tier = user.get("subscription", {}).get("tier", "unknown")
#         tier_folder = f"{users_folder}/by_tier/{tier}"
        
#         # Create tier folder if it doesn't exist
#         try:
#             dbutils.fs.mkdirs(tier_folder)
#         except:
#             pass  # Folder exists
        
#         # Save user to appropriate tier folder
#         user_file = f"{tier_folder}/{user['user_id']}.json"
#         dbutils.fs.put(user_file, json.dumps(user), overwrite=True)

# print("✅ Created partitioned user data by subscription tier")

# List the created structure
print("\n📂 Folder structure created:")
display(dbutils.fs.ls(users_folder))


✅ Created folder: /Volumes/pavan_naidu/json/raw_data/users
✅ Saved JSON data to: /Volumes/pavan_naidu/json/raw_data/users/user_data.json
✅ Saved JSONL data to: /Volumes/pavan_naidu/json/raw_data/users/user_data.jsonl

📂 Folder structure created:


[FileInfo(path='/Volumes/pavan_naidu/json/raw_data/users/user_data.json', name='user_data.json', size=1158297, modificationTime=1759332338000),
 FileInfo(path='/Volumes/pavan_naidu/json/raw_data/users/user_data.jsonl', name='user_data.jsonl', size=1157296, modificationTime=1759332338000)]

## 2. Creating Tables with VARIANT Data Type

Let's create tables to store our JSON data using the VARIANT type.


In [6]:
# Drop tables if they exist
spark.sql("DROP TABLE IF EXISTS users_variant")
spark.sql("DROP TABLE IF EXISTS users_extracted")

# Create a table with VARIANT column for user data
# Enable Delta feature for column defaults
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS users_variant (
        user_id STRING,
        username STRING,
        email STRING,
        user_data VARIANT,
        ingestion_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )
    USING DELTA
    TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported')
    COMMENT 'Table storing user data in VARIANT column'
""")

print("✅ Created table: users_variant with column defaults enabled")

# Create another table with extracted fields for performance comparison
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS users_extracted (
        user_id STRING,
        username STRING,
        email STRING,
        name STRING,
        age INT,
        city STRING,
        country STRING,
        subscription_tier STRING,
        user_data VARIANT,
        ingestion_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
    )
    USING DELTA
    TBLPROPERTIES('delta.feature.allowColumnDefaults' = 'supported')
    COMMENT 'Table with commonly accessed fields extracted from VARIANT'
""")

print("✅ Created table: users_extracted")

# Show table schemas
print("\nTable Schemas:")
spark.sql("DESCRIBE TABLE users_variant").show()
spark.sql("DESCRIBE TABLE users_extracted").show()


✅ Created table: users_variant with column defaults enabled
✅ Created table: users_extracted

Table Schemas:
+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|            user_id|   string|   NULL|
|           username|   string|   NULL|
|              email|   string|   NULL|
|          user_data|  variant|   NULL|
|ingestion_timestamp|timestamp|   NULL|
+-------------------+---------+-------+

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|            user_id|   string|   NULL|
|           username|   string|   NULL|
|              email|   string|   NULL|
|               name|   string|   NULL|
|                age|      int|   NULL|
|               city|   string|   NULL|
|            country|   string|   NULL|
|  subscription_tier|   string|   NULL|
|          user_data|  variant|   NULL|
|ingestion_timestamp|timestamp|   NULL|
+---------

## 3. Loading Data into VARIANT Columns

Now let's load our JSON data into the tables using different methods.


In [7]:
# Method 1: Load from JSON file using PARSE_JSON
spark.sql(f"""
    INSERT INTO users_variant (user_id, username, email, user_data)
    SELECT 
        PARSE_JSON(value):user_id::String as user_id,
        PARSE_JSON(value):username as username,
        PARSE_JSON(value):email as email,
        PARSE_JSON(value) as user_data
    FROM (
        SELECT value 
        FROM text.`{jsonl_file_path}`
    )
""")

count = spark.sql("SELECT COUNT(*) FROM users_variant").collect()[0][0]
print(f"✅ Loaded {count} records into users_variant using PARSE_JSON")

# Show sample data
print("\nSample data from users_variant:")
spark.sql("SELECT * FROM users_variant LIMIT 5").show(truncate=False)


✅ Loaded 1000 records into users_variant using PARSE_JSON

Sample data from users_variant:


+------------------------------------+------------+------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
# Method 2: Load data with extracted fields for optimized queries
spark.sql(f"""
    INSERT INTO users_extracted 
    SELECT 
        PARSE_JSON(value):user_id as user_id,
        PARSE_JSON(value):username as username,
        PARSE_JSON(value):email as email,
        PARSE_JSON(value):name as name,
        CAST(PARSE_JSON(value):age AS INT) as age,
        PARSE_JSON(value):address.city as city,
        PARSE_JSON(value):address.country as country,
        PARSE_JSON(value):subscription.tier as subscription_tier,
        PARSE_JSON(value) as user_data,
        CURRENT_TIMESTAMP() as ingestion_timestamp
    FROM (
        SELECT value 
        FROM text.`{jsonl_file_path}`
    )
""")

count = spark.sql("SELECT COUNT(*) FROM users_extracted").collect()[0][0]
print(f"✅ Loaded {count} records into users_extracted with extracted fields")

# Show sample
print("\nSample data from users_extracted (with extracted fields):")
spark.sql("SELECT user_id, name, age, city, subscription_tier FROM users_extracted LIMIT 5").show(truncate=False)


✅ Loaded 1000 records into users_extracted with extracted fields

Sample data from users_extracted (with extracted fields):


+------------------------------------+-----------------+---+-------------------+-----------------+
|user_id                             |name             |age|city               |subscription_tier|
+------------------------------------+-----------------+---+-------------------+-----------------+
|60cabef2-6c60-43d6-aa55-df67edcee606|Melanie Gutierrez|79 |Butlerton          |NULL             |
|7e8614cf-da0b-4580-89e3-dbb8880eb67c|Kelsey Welch     |33 |Port Diana         |NULL             |
|bc0268cb-3954-4f4b-99ab-03dff42621f3|William Butler   |38 |Port Haroldton     |NULL             |
|5fd673b7-34cd-413f-8495-d3263e375133|Kayla Hall       |23 |South Stephenville |NULL             |
|3d2a259a-c14a-4ef6-a7e5-02bd95f532a8|Megan Moon       |62 |North Mackenzieview|NULL             |
+------------------------------------+-----------------+---+-------------------+-----------------+



## VARIANT Query Notation Reference

Before diving into examples, here's the correct notation for querying VARIANT data:

### Basic Notation Rules:

1. **Field Access**: Use `:` to access top-level fields
   ```sql
   column_name:field_name
   ```

2. **Subfield Access**: Use `.` to access nested fields
   ```sql
   column_name:field_name.subfield_name
   ```

3. **Array Element Access**: Use `[n]` to access array elements by index
   ```sql
   column_name:array_name[0]  -- First element (0-indexed)
   column_name:array_name[1]  -- Second element
   ```

### Examples:

```sql
-- Correct VARIANT notation examples
SELECT 
  user_data:name as name,                           -- Field access
  user_data:age as age,                             -- Field access
  user_data:profile.bio as bio,                     -- Subfield access
  user_data:address.city as city,                   -- Nested subfield
  user_data:address.coordinates.latitude as lat,    -- Deeply nested subfield
  user_data:interests[0] as first_interest,         -- Array element access
  user_data:profile.skills[1] as second_skill       -- Array element in nested object
FROM users_variant;
```


## 4. Querying VARIANT Data - Basic Access Patterns

Let's explore different ways to access and query data stored in VARIANT columns.


In [9]:
# Accessing nested fields using correct VARIANT notation
print("=== Accessing Nested Fields ===\n")
print("VARIANT Query Notation:")
print("- Field access: column_name:field_name")
print("- Subfield access: column_name:field_name.subfield_name") 
print("- Array access: column_name:array_name[n]")
print()

spark.sql("""
    SELECT 
        user_id,
        user_data:name as name,
        user_data:age as age,
        user_data:profile.bio as bio,
        user_data:profile.occupation as occupation,
        user_data:address.city as city,
        user_data:address.coordinates.latitude as latitude,
        user_data:address.coordinates.longitude as longitude
    FROM users_variant
    LIMIT 5
""").show(truncate=False)


=== Accessing Nested Fields ===

VARIANT Query Notation:
- Field access: column_name:field_name
- Subfield access: column_name:field_name.subfield_name
- Array access: column_name:array_name[n]



+------------------------------------+-------------------+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+---------------------+-----------+----------+
|user_id                             |name               |age|bio                                                                                                                                                                               |occupation                          |city                 |latitude   |longitude |
+------------------------------------+-------------------+---+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+---------------------+-----------+----------+
|60cabef2-6c60-43d6-aa55-df6

In [10]:
# Handling NULL values in VARIANT with is_variant_null function
print("\n=== is_variant_null Function Examples ===\n")

# Simple example: is_variant_null vs IS NULL
print("Basic example - difference between is_variant_null() and IS NULL:")
spark.sql("""
    SELECT 
        parse_json('{"name": "John", "age": null}') as data,
        is_variant_null(parse_json('{"name": "John", "age": null}'):age) as age_is_variant_null,
        parse_json('{"name": "John", "age": null}'):age IS NULL as age_is_null
""").show()

# Filtering with is_variant_null
print("\nFiltering records where city is variant null:")
spark.sql("""
    SELECT 
        data:name as name,
        data:city as city
    FROM VALUES
        (parse_json('{"name": "John", "city": null}')),
        (parse_json('{"name": "Jane", "city": "NYC"}')),
        (parse_json('{"name": "Bob", "city": null}'))
    AS T(data)
    WHERE is_variant_null(data:city) = true
""").show()

print("\n✅ Simple examples of is_variant_null function!")
print("📚 Reference: https://docs.databricks.com/gcp/en/sql/language-manual/functions/is_variant_null")



=== is_variant_null Function Examples ===

Basic example - difference between is_variant_null() and IS NULL:


+--------------------+-------------------+-----------+
|                data|age_is_variant_null|age_is_null|
+--------------------+-------------------+-----------+
|{"age":null,"name...|               true|      false|
+--------------------+-------------------+-----------+


Filtering records where city is variant null:
+------+----+
|  name|city|
+------+----+
|"John"|null|
| "Bob"|null|
+------+----+


✅ Simple examples of is_variant_null function!
📚 Reference: https://docs.databricks.com/gcp/en/sql/language-manual/functions/is_variant_null


In [11]:
# Accessing array elements within VARIANT
print("=== Accessing Array Elements in VARIANT ===\n")

spark.sql("""
    SELECT 
        user_id,
        username,
        user_data:profile.interests[0] as first_interest,
        user_data:profile.interests[1] as second_interest,
        SIZE(CAST(user_data:profile.interests AS ARRAY<STRING>)) as total_interests,
        user_data:profile.skills as all_skills
    FROM users_variant
    WHERE NOT IS_VARIANT_NULL(user_data:profile.interests)
    LIMIT 5
""").show(truncate=False)

# Using VARIANT_EXPLODE with arrays in VARIANT
print("\n=== Exploding Arrays from VARIANT ===\n")

spark.sql("""
    SELECT 
        user_id,
        username,
        value as interest
    FROM users_variant,
    LATERAL VARIANT_EXPLODE(user_data:profile.interests)
    LIMIT 10
""").show(truncate=False)


=== Accessing Array Elements in VARIANT ===



+------------------------------------+------------+--------------+---------------+---------------+----------------------------------------------------------------------+
|user_id                             |username    |first_interest|second_interest|total_interests|all_skills                                                            |
+------------------------------------+------------+--------------+---------------+---------------+----------------------------------------------------------------------+
|60cabef2-6c60-43d6-aa55-df67edcee606|hcameron    |"strong"      |"happy"        |4              |[]                                                                    |
|7e8614cf-da0b-4580-89e3-dbb8880eb67c|renee03     |"notice"      |"lead"         |2              |["Engineer, site","Architectural technologist","Field trials officer"]|
|bc0268cb-3954-4f4b-99ab-03dff42621f3|liuamanda   |"strong"      |"culture"      |4              |[]                                                  

+------------------------------------+---------+---------+
|user_id                             |username |interest |
+------------------------------------+---------+---------+
|60cabef2-6c60-43d6-aa55-df67edcee606|hcameron |"strong" |
|60cabef2-6c60-43d6-aa55-df67edcee606|hcameron |"happy"  |
|60cabef2-6c60-43d6-aa55-df67edcee606|hcameron |"kind"   |
|60cabef2-6c60-43d6-aa55-df67edcee606|hcameron |"wonder" |
|7e8614cf-da0b-4580-89e3-dbb8880eb67c|renee03  |"notice" |
|7e8614cf-da0b-4580-89e3-dbb8880eb67c|renee03  |"lead"   |
|bc0268cb-3954-4f4b-99ab-03dff42621f3|liuamanda|"strong" |
|bc0268cb-3954-4f4b-99ab-03dff42621f3|liuamanda|"culture"|
|bc0268cb-3954-4f4b-99ab-03dff42621f3|liuamanda|"book"   |
|bc0268cb-3954-4f4b-99ab-03dff42621f3|liuamanda|"or"     |
+------------------------------------+---------+---------+



## 5. Type Casting and Conversion with VARIANT

VARIANT columns can be cast to specific types when needed.


In [12]:
# Type casting VARIANT fields
print("=== Type Casting VARIANT Fields ===\n")

spark.sql("""
    SELECT 
        user_id,
        user_data:age as age_variant,
        CAST(user_data:age AS INT) as age_int,
        CAST(user_data:created_at AS TIMESTAMP) as created_timestamp,
        CAST(user_data:preferences.newsletter AS BOOLEAN) as newsletter_bool,
        CAST(user_data:metrics.login_count AS INT) as login_count_int
    FROM users_variant
    LIMIT 5
""").show()

# Using TRY_CAST for safe casting (handles nulls gracefully)
print("\n=== Safe Casting with TRY_CAST ===\n")

spark.sql("""
    SELECT 
        user_id,
        TRY_CAST(user_data:age AS INT) as age,
        TRY_CAST(user_data:subscription.start_date AS DATE) as subscription_date,
        TRY_CAST(user_data:metrics.posts_created AS INT) as posts_count,
        TRY_CAST(user_data:phone AS STRING) as phone_number,
        TRY_CAST(user_data:last_login AS TIMESTAMP) as last_login_time
    FROM users_variant
    LIMIT 5
""").show()


=== Type Casting VARIANT Fields ===



+--------------------+-----------+-------+--------------------+---------------+---------------+
|             user_id|age_variant|age_int|   created_timestamp|newsletter_bool|login_count_int|
+--------------------+-----------+-------+--------------------+---------------+---------------+
|60cabef2-6c60-43d...|         79|     79|2023-04-26 15:38:...|          false|           NULL|
|7e8614cf-da0b-458...|         33|     33|2020-11-26 23:12:...|           true|           NULL|
|bc0268cb-3954-4f4...|         38|     38|2024-06-26 07:18:...|           true|           NULL|
|5fd673b7-34cd-413...|         23|     23|2025-08-05 18:41:...|           true|           NULL|
|3d2a259a-c14a-4ef...|         62|     62|2024-12-12 19:18:...|           true|           NULL|
+--------------------+-----------+-------+--------------------+---------------+---------------+


=== Safe Casting with TRY_CAST ===



+--------------------+---+-----------------+-----------+----------------+--------------------+
|             user_id|age|subscription_date|posts_count|    phone_number|     last_login_time|
+--------------------+---+-----------------+-----------+----------------+--------------------+
|60cabef2-6c60-43d...| 79|             NULL|       NULL|            NULL|2025-09-25 21:52:...|
|7e8614cf-da0b-458...| 33|             NULL|       NULL|            NULL|2025-09-27 22:31:...|
|bc0268cb-3954-4f4...| 38|             NULL|       NULL|            NULL|2025-09-27 19:08:...|
|5fd673b7-34cd-413...| 23|             NULL|       NULL|819-579-1555x104|2025-09-21 15:57:...|
|3d2a259a-c14a-4ef...| 62|             NULL|       NULL|            NULL|2025-09-11 20:07:...|
+--------------------+---+-----------------+-----------+----------------+--------------------+



## 6. VARIANT Functions and Operations

Databricks provides several functions specifically for working with VARIANT data.


In [13]:
# VARIANT_GET function to safely extract values
print("=== Using VARIANT_GET Function ===\n")

spark.sql("""
    SELECT 
        user_id,
        VARIANT_GET(user_data, '$.name', 'STRING') as name,
        VARIANT_GET(user_data, '$.age', 'INT') as age,
        VARIANT_GET(user_data, '$.profile.interests', 'ARRAY<STRING>') as interests,
        VARIANT_GET(user_data, '$.non_existent_field', 'STRING') as missing_field
    FROM users_variant
    LIMIT 3
""").show(truncate=False)


=== Using VARIANT_GET Function ===



+------------------------------------+-----------------+---+-----------------------------+-------------+
|user_id                             |name             |age|interests                    |missing_field|
+------------------------------------+-----------------+---+-----------------------------+-------------+
|60cabef2-6c60-43d6-aa55-df67edcee606|Melanie Gutierrez|79 |[strong, happy, kind, wonder]|NULL         |
|7e8614cf-da0b-4580-89e3-dbb8880eb67c|Kelsey Welch     |33 |[notice, lead]               |NULL         |
|bc0268cb-3954-4f4b-99ab-03dff42621f3|William Butler   |38 |[strong, culture, book, or]  |NULL         |
+------------------------------------+-----------------+---+-----------------------------+-------------+



In [14]:
# TO_JSON and PARSE_JSON functions
print("=== Converting between VARIANT and JSON strings ===\n")

spark.sql("""
    SELECT 
        user_id,
        TO_JSON(user_data) as json_string,
        LENGTH(TO_JSON(user_data)) as json_length,
        PARSE_JSON(TO_JSON(user_data:address)) as address_variant
    FROM users_variant
    LIMIT 2
""").show(truncate=False)

# SCHEMA_OF_VARIANT to discover schema
print("\n=== Discovering Schema of VARIANT Data ===\n")

spark.sql("""
    SELECT 
        SCHEMA_OF_VARIANT(user_data) as inferred_schema,
        count(*) as count
    FROM users_variant
    GROUP BY SCHEMA_OF_VARIANT(user_data)
    ORDER BY count(*) DESC
    LIMIT 5
""").show(truncate=False)


=== Converting between VARIANT and JSON strings ===



+------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------------

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|inferred_schema                                                                                                                                                                                                                                                                                      

In [15]:
# SCHEMA_OF_VARIANT_AGG 
print("\n=== SCHEMA_OF_VARIANT_AGG Examples ===\n")

# Example 1: Get combined schema of all VARIANT values
print("1. Combined schema of all VARIANT values:")
spark.sql("""
    SELECT SCHEMA_OF_VARIANT_AGG(user_data) as  
    FROM users_variant
""").show(truncate=False)

# Example 2: Get schema grouped by subscription tier
print("\n2. Schema grouped by subscription tier:")
spark.sql("""
    SELECT 
        CAST(user_data:subscription.tier AS STRING) as subscription_tier,
        SCHEMA_OF_VARIANT_AGG(user_data) as tier_schema,
        COUNT(*) as record_count
    FROM users_variant 
    WHERE NOT IS_VARIANT_NULL(user_data:subscription.tier)
    GROUP BY CAST(user_data:subscription.tier AS STRING)
    ORDER BY record_count DESC
""").show(truncate=False)


=== SCHEMA_OF_VARIANT_AGG Examples ===

1. Combined schema of all VARIANT values:


+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|as                                                    

+-----------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|subscription_tier|tier_

## 7. Filtering and Conditional Logic with VARIANT

Let's explore how to filter and apply conditions on VARIANT data.


In [16]:
# Filtering based on VARIANT field values
print("=== Filtering on VARIANT Fields ===\n")

spark.sql("""
    SELECT 
        user_id,
        user_data:name as name,
        user_data:age as age,
        user_data:address.country as country
    FROM users_variant
    WHERE CAST(user_data:age AS INT) > 30
        AND CAST(user_data:preferences.newsletter AS BOOLEAN) = true
        AND NOT IS_VARIANT_NULL(user_data:address.country)
    LIMIT 5
""").show()

# Complex filtering with nested conditions
print("\n=== Complex Filtering with Nested VARIANT Fields ===\n")

spark.sql("""
    SELECT 
        user_id,
        user_data:name as name,
        user_data:age as age,
        CAST(user_data:subscription.tier AS STRING) as subscription_tier,
        user_data:metrics.login_count as login_count
    FROM users_variant
    WHERE NOT IS_VARIANT_NULL(user_data:subscription.tier)
        AND NOT IS_VARIANT_NULL(user_data:metrics.login_count)
        AND CAST(user_data:age AS INT) BETWEEN 25 AND 60
        AND CAST(user_data:metrics.login_count AS INT) > 10
    ORDER BY CAST(user_data:metrics.login_count AS INT) DESC
    LIMIT 5
""").show()


=== Filtering on VARIANT Fields ===



+--------------------+--------------------+---+-------------+
|             user_id|                name|age|      country|
+--------------------+--------------------+---+-------------+
|7e8614cf-da0b-458...|      "Kelsey Welch"| 33|  "Greenland"|
|bc0268cb-3954-4f4...|    "William Butler"| 38| "Guadeloupe"|
|3d2a259a-c14a-4ef...|        "Megan Moon"| 62|  "Argentina"|
|bfa77ec9-1930-463...|       "Alex Butler"| 55|"New Zealand"|
|6a8246d6-9092-477...|"Christopher Wilson"| 67| "Luxembourg"|
+--------------------+--------------------+---+-------------+


=== Complex Filtering with Nested VARIANT Fields ===



+--------------------+----------------+---+-----------------+-----------+
|             user_id|            name|age|subscription_tier|login_count|
+--------------------+----------------+---+-----------------+-----------+
|7b450bc2-2d49-45e...|   "Tammy Bauer"| 44|            basic|       1000|
|f8cc59c4-e654-4e8...|     "John Moon"| 42|             free|        996|
|7440daa1-42c7-473...|"Robin Gallegos"| 41|          premium|        995|
|20fc3134-1ad6-40e...| "Nicole Ortega"| 38|             free|        992|
|79148f8e-ea6e-475...|"Jennifer Lucas"| 45|          premium|        987|
+--------------------+----------------+---+-----------------+-----------+



In [17]:
# Using CASE statements with VARIANT
print("=== CASE Statements with VARIANT ===\n")

spark.sql("""
    SELECT 
        user_id,
        user_data:name as name,
        user_data:age as age,
        CAST(user_data:subscription.tier AS STRING) as subscription_tier,
        CASE 
            WHEN CAST(user_data:age AS INT) < 25 THEN 'Young'
            WHEN CAST(user_data:age AS INT) BETWEEN 25 AND 50 THEN 'Adult'
            WHEN CAST(user_data:age AS INT) > 50 THEN 'Senior'
            ELSE 'Unknown'
        END as age_group,
        CASE 
            WHEN CAST(user_data:subscription.tier AS STRING) = 'free' THEN 'Free User'
            WHEN CAST(user_data:subscription.tier AS STRING) = 'basic' THEN 'Basic User'
            WHEN CAST(user_data:subscription.tier AS STRING) = 'premium' THEN 'Premium User'
            WHEN CAST(user_data:subscription.tier AS STRING) = 'enterprise' THEN 'Enterprise User'
            ELSE 'No Subscription'
        END as user_category
    FROM users_variant
    LIMIT 10
""").show(truncate=False)


=== CASE Statements with VARIANT ===



+------------------------------------+--------------------+---+-----------------+---------+---------------+
|user_id                             |name                |age|subscription_tier|age_group|user_category  |
+------------------------------------+--------------------+---+-----------------+---------+---------------+
|60cabef2-6c60-43d6-aa55-df67edcee606|"Melanie Gutierrez" |79 |NULL             |Senior   |No Subscription|
|7e8614cf-da0b-4580-89e3-dbb8880eb67c|"Kelsey Welch"      |33 |NULL             |Adult    |No Subscription|
|bc0268cb-3954-4f4b-99ab-03dff42621f3|"William Butler"    |38 |NULL             |Adult    |No Subscription|
|5fd673b7-34cd-413f-8495-d3263e375133|"Kayla Hall"        |23 |NULL             |Young    |No Subscription|
|3d2a259a-c14a-4ef6-a7e5-02bd95f532a8|"Megan Moon"        |62 |NULL             |Senior   |No Subscription|
|ae87650d-4989-4f66-9601-2a93ec8f1006|"Michelle Meyer"    |41 |NULL             |Adult    |No Subscription|
|5542230a-ba1e-40ad-9531-2f5

## 8. Aggregations with VARIANT Data

Performing aggregations on data stored in VARIANT columns.


In [18]:
# Aggregating VARIANT data
print("=== Aggregations on VARIANT Data ===\n")

spark.sql("""
    SELECT 
        COUNT(*) as user_count,
        AVG(CAST(user_data:age AS INT)) as avg_age,
        MIN(CAST(user_data:age AS INT)) as min_age,
        MAX(CAST(user_data:age AS INT)) as max_age
    FROM users_variant
""").show()

# Aggregating by subscription tier
print("\n=== Subscription Tier Aggregations ===\n")

spark.sql("""
    SELECT 
        CAST(user_data:subscription.tier AS STRING) as subscription_tier,
        COUNT(*) as user_count,
        AVG(CAST(user_data:age AS INT)) as avg_age,
        AVG(CAST(user_data:metrics.login_count AS INT)) as avg_login_count,
        AVG(CAST(user_data:metrics.posts_created AS INT)) as avg_posts
    FROM users_variant
    WHERE NOT IS_VARIANT_NULL(user_data:subscription.tier)
    GROUP BY CAST(user_data:subscription.tier AS STRING)
    ORDER BY user_count DESC
""").show()


=== Aggregations on VARIANT Data ===



+----------+-------+-------+-------+
|user_count|avg_age|min_age|max_age|
+----------+-------+-------+-------+
|      1000|  49.03|     18|     80|
+----------+-------+-------+-------+


=== Subscription Tier Aggregations ===



+-----------------+----------+------------------+------------------+------------------+
|subscription_tier|user_count|           avg_age|   avg_login_count|         avg_posts|
+-----------------+----------+------------------+------------------+------------------+
|             NULL|       601|48.302828618968384|              NULL|              NULL|
|       enterprise|       107|  47.6822429906542| 565.0841121495328| 253.6355140186916|
|            basic|        99| 50.91919191919192|494.64646464646466|             252.0|
|          premium|        98| 50.55102040816327| 503.3061224489796|236.87755102040816|
|             free|        95| 51.61052631578947| 546.1578947368421|248.05263157894737|
+-----------------+----------+------------------+------------------+------------------+



In [19]:
# Complex aggregations with array data
print("=== Aggregating Array Data in VARIANT ===\n")

spark.sql("""
    SELECT 
        AVG(SIZE(CAST(user_data:profile.interests AS ARRAY<STRING>))) as avg_interests_per_user,
        MAX(SIZE(CAST(user_data:profile.interests AS ARRAY<STRING>))) as max_interests,
        MIN(SIZE(CAST(user_data:profile.interests AS ARRAY<STRING>))) as min_interests
    FROM users_variant
    WHERE NOT IS_VARIANT_NULL(user_data:profile.interests)
""").show()

# Aggregating skills data
print("\n=== Skills Aggregation ===\n")

spark.sql("""
    WITH user_skills AS (
        SELECT 
            user_id,
            user_data:name as name,
            SIZE(CAST(user_data:profile.skills AS ARRAY<STRING>)) as skill_count,
            user_data:profile.skills as skills
        FROM users_variant
        WHERE NOT IS_VARIANT_NULL(user_data:profile.skills)
    )
    SELECT 
        COUNT(*) as users_with_skills,
        AVG(skill_count) as avg_skills_per_user,
        MAX(skill_count) as max_skills,
        MIN(skill_count) as min_skills
    FROM user_skills
""").show()


=== Aggregating Array Data in VARIANT ===



+----------------------+-------------+-------------+
|avg_interests_per_user|max_interests|min_interests|
+----------------------+-------------+-------------+
|                 3.005|            5|            1|
+----------------------+-------------+-------------+


=== Skills Aggregation ===



+-----------------+-------------------+----------+----------+
|users_with_skills|avg_skills_per_user|max_skills|min_skills|
+-----------------+-------------------+----------+----------+
|             1000|              1.498|         3|         0|
+-----------------+-------------------+----------+----------+



## 9. Window Functions with VARIANT

Using window functions to analyze VARIANT data.


In [20]:
# Window functions with VARIANT data
print("=== Window Functions on VARIANT Data ===\n")

spark.sql("""
    SELECT 
        user_id,
        user_data:name as name,
        user_data:age as age,
        CAST(user_data:address.country AS STRING) as country,
        CAST(user_data:metrics.login_count AS INT) as login_count,
        RANK() OVER (PARTITION BY CAST(user_data:address.country AS STRING) ORDER BY CAST(user_data:age AS INT) DESC) as age_rank_in_country,
        DENSE_RANK() OVER (ORDER BY CAST(user_data:metrics.login_count AS INT) DESC) as login_rank,
        AVG(CAST(user_data:age AS INT)) OVER (PARTITION BY CAST(user_data:address.country AS STRING)) as country_avg_age
    FROM users_variant
    WHERE NOT IS_VARIANT_NULL(user_data:metrics.login_count)
        AND NOT IS_VARIANT_NULL(user_data:address.country)
    ORDER BY country, age_rank_in_country
    LIMIT 15
""").show(truncate=False)

# Running totals and cumulative stats
print("\n=== Cumulative Statistics with VARIANT ===\n")

spark.sql("""
    WITH user_activity AS (
        SELECT 
            user_id,
            user_data:name as name,
            CAST(user_data:created_at AS TIMESTAMP) as created_time,
            CAST(user_data:metrics.login_count AS INT) as login_count,
            CAST(user_data:metrics.posts_created AS INT) as posts_created
        FROM users_variant
        WHERE NOT IS_VARIANT_NULL(user_data:metrics.login_count)
            AND NOT IS_VARIANT_NULL(user_data:created_at)
    )
    SELECT 
        user_id,
        name,
        created_time,
        login_count,
        posts_created,
        AVG(login_count) OVER (
            ORDER BY created_time 
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ) as moving_avg_logins,
        MAX(login_count) OVER (
            ORDER BY created_time
        ) as max_logins_so_far
    FROM user_activity
    ORDER BY created_time
    LIMIT 10
""").show(truncate=False)


=== Window Functions on VARIANT Data ===



+------------------------------------+-----------------+---+-----------+-----------+-------------------+----------+-----------------+
|user_id                             |name             |age|country    |login_count|age_rank_in_country|login_rank|country_avg_age  |
+------------------------------------+-----------------+---+-----------+-----------+-------------------+----------+-----------------+
|19b9569b-dd19-430c-a7a2-e1de34ae175b|"Dustin Jackson" |58 |Afghanistan|NULL       |1                  |329       |34.0             |
|b2a7fcab-e684-48ea-a1dc-2887dfd1d6fe|"Amber King"     |25 |Afghanistan|NULL       |2                  |329       |34.0             |
|5f89b9e7-81dd-4ab7-9176-763792b129dc|"Annette Harris" |19 |Afghanistan|169        |3                  |277       |34.0             |
|d9292b92-c781-44f7-8bca-90117e91b149|"Penny Jimenez"  |80 |Albania    |226        |1                  |262       |49.0             |
|1abb10f4-8a3b-4c25-9296-e25e2ae025c0|"Stacy Baker"    |52 |Al

+------------------------------------+---------------------+--------------------------+-----------+-------------+-----------------+-----------------+
|user_id                             |name                 |created_time              |login_count|posts_created|moving_avg_logins|max_logins_so_far|
+------------------------------------+---------------------+--------------------------+-----------+-------------+-----------------+-----------------+
|5e246fcf-48e2-4d6e-9c92-aa67f74730f1|"Christopher Spencer"|2020-10-03 01:15:23.399003|NULL       |NULL         |NULL             |NULL             |
|a31d8dc6-3dfd-4fe4-97ad-af1fdd601873|"Michael Hawkins"    |2020-10-04 13:54:58.831194|NULL       |NULL         |NULL             |NULL             |
|7eeb0388-8a61-49db-ad3c-fa128d592af9|"Rachel Moore"       |2020-10-05 05:43:50.972837|NULL       |NULL         |NULL             |NULL             |
|5f691341-d14d-457b-8177-9d6b0d61f277|"Jeffrey Gonzalez"   |2020-10-05 10:17:28.872737|154        |1

## 10. JOINs with VARIANT Data

Demonstrating how to join tables containing VARIANT columns.


In [21]:
# Self-join on VARIANT data
print("=== Self-Join on VARIANT Data ===\n")

spark.sql("""
    SELECT 
        u1.user_id as user1_id,
        u1.user_data:name as user1_name,
        CAST(u1.user_data:address.city AS STRING) as user1_city,
        u2.user_id as user2_id,
        u2.user_data:name as user2_name,
        CAST(u2.user_data:address.city AS STRING) as user2_city
    FROM users_variant u1
    JOIN users_variant u2
        ON CAST(u1.user_data:address.city AS STRING) = CAST(u2.user_data:address.city AS STRING)
        AND u1.user_id < u2.user_id
    WHERE NOT IS_VARIANT_NULL(u1.user_data:address.city)
    LIMIT 10
""").show(truncate=False)

# Join with extracted table for performance
print("\n=== Join with Extracted Table for Performance ===\n")

spark.sql("""
    WITH user_analysis AS (
        SELECT 
            v.user_id,
            v.user_data:name as name,
            v.user_data:age as age,
            CAST(v.user_data:address.city AS STRING) as city,
            CAST(v.user_data:subscription.tier AS STRING) as subscription_tier,
            e.name as extracted_name,
            e.age as extracted_age,
            e.city as extracted_city
        FROM users_variant v
        LEFT JOIN users_extracted e
            ON v.user_id = e.user_id
        WHERE NOT IS_VARIANT_NULL(v.user_data:subscription.tier)
    )
    SELECT * FROM user_analysis
    WHERE name IS NOT NULL
    LIMIT 5
""").show(truncate=False)


=== Self-Join on VARIANT Data ===



+------------------------------------+-----------------+--------------+------------------------------------+-------------------+--------------+
|user1_id                            |user1_name       |user1_city    |user2_id                            |user2_name         |user2_city    |
+------------------------------------+-----------------+--------------+------------------------------------+-------------------+--------------+
|8fe88a32-0792-40b8-8ec8-9f77fd002dc5|"Sara Ferguson"  |Garrisontown  |e79f67b6-1592-4651-80fc-14e26f022b98|"Yesenia Luna"     |Garrisontown  |
|a4d44be2-4326-40cc-aec7-9b55915fdb86|"Derrick Freeman"|Lake James    |ad6c3c33-7e70-412f-85d5-e52ca141bdcf|"Emily Pacheco"    |Lake James    |
|d9f6f0c6-46cc-4d29-bca1-f0fc2ef2c22c|"Andrew Costa"   |Jonesmouth    |f45377ef-0c36-489f-9fe8-5f445ce16502|"Victoria Leon"    |Jonesmouth    |
|69dd65b0-b70b-44fd-9421-087db40d5b05|"Kevin Luna"     |Davidhaven    |d9b48fd7-7d28-42e4-b81b-340d96e1ef02|"Carl Payne"       |Davidhav

+------------------------------------+-------------------+---+-------------------+-----------------+-----------------+-------------+-------------------+
|user_id                             |name               |age|city               |subscription_tier|extracted_name   |extracted_age|extracted_city     |
+------------------------------------+-------------------+---+-------------------+-----------------+-----------------+-------------+-------------------+
|60cabef2-6c60-43d6-aa55-df67edcee606|"Melanie Gutierrez"|79 |Butlerton          |NULL             |Melanie Gutierrez|79           |Butlerton          |
|7e8614cf-da0b-4580-89e3-dbb8880eb67c|"Kelsey Welch"     |33 |Port Diana         |NULL             |Kelsey Welch     |33           |Port Diana         |
|bc0268cb-3954-4f4b-99ab-03dff42621f3|"William Butler"   |38 |Port Haroldton     |NULL             |William Butler   |38           |Port Haroldton     |
|5fd673b7-34cd-413f-8495-d3263e375133|"Kayla Hall"       |23 |South Stephenville |

## 11. Advanced VARIANT Use Cases

Let's explore some advanced use cases and patterns.


In [22]:
# Dynamic schema evolution handling
print("=== Demonstrating Schema Evolution with VARIANT ===\n")

# The data we generated already simulates schema evolution
# Let's query to show how VARIANT handles different schema versions seamlessly

print("Schema Evolution in our user data:")
print("- First 30% of users: Basic schema")
print("- Next 30% of users: Added social_media fields")
print("- Last 40% of users: Added subscription and metrics fields\n")

# Query showing all schema versions work together
spark.sql("""
    SELECT 
        user_id,
        user_data:name as name,
        user_data:email as email,
        user_data:phone as phone,  -- Optional field
        user_data:social_media.twitter as twitter,  -- Added in v2
        user_data:subscription.tier as subscription_tier,  -- Added in v3
        user_data:metrics.login_count as login_count  -- Added in v3
    FROM users_variant
    WHERE user_id IN (
        (SELECT user_id FROM users_variant LIMIT 1 OFFSET 100)
        UNION ALL
        (SELECT user_id FROM users_variant LIMIT 1 OFFSET 500)
        UNION ALL
        (SELECT user_id FROM users_variant LIMIT 1 OFFSET 900)
    )
""").show(truncate=False)

print("\n✅ VARIANT seamlessly handles schema evolution - old and new fields coexist!")


=== Demonstrating Schema Evolution with VARIANT ===

Schema Evolution in our user data:
- First 30% of users: Basic schema
- Next 30% of users: Added social_media fields
- Last 40% of users: Added subscription and metrics fields



+------------------------------------+-------------------+-----------------------------+-----+----------------+-----------------+-----------+
|user_id                             |name               |email                        |phone|twitter         |subscription_tier|login_count|
+------------------------------------+-------------------+-----------------------------+-----+----------------+-----------------+-----------+
|1740eff5-bd8f-42a1-9bdf-3621cdf13b15|"Michael Chen"     |"jeffrey60@example.com"      |NULL |NULL            |NULL             |NULL       |
|d350e4d7-b2f2-43a6-bbf1-81b29d0b9b45|"Christopher Scott"|"lori33@example.net"         |NULL |null            |NULL             |NULL       |
|f5c1383a-2286-44e1-94be-fc112536bf98|"Angela Lee"       |"amandacontreras@example.org"|NULL |"@melindaharris"|"basic"          |30         |
+------------------------------------+-------------------+-----------------------------+-----+----------------+-----------------+-----------+


✅ VA

In [23]:
# Pivoting VARIANT data
print("=== Pivoting VARIANT Data ===\n")

spark.sql("""
    WITH subscription_counts AS (
        SELECT 
            CAST(user_data:address.country AS STRING) as country,
            CAST(user_data:subscription.tier AS STRING) as subscription_tier,
            COUNT(*) as user_count
        FROM users_variant
        WHERE NOT IS_VARIANT_NULL(user_data:subscription.tier)
            AND NOT IS_VARIANT_NULL(user_data:address.country)
        GROUP BY country, subscription_tier
    )
    SELECT * FROM subscription_counts
    PIVOT (
        SUM(user_count)
        FOR subscription_tier IN ('free', 'basic', 'premium', 'enterprise')
    )
    ORDER BY country
    LIMIT 10
""").show()

# Creating materialized views from VARIANT data
print("\n=== Creating Structured View from VARIANT ===\n")

spark.sql("""
    CREATE OR REPLACE TEMPORARY VIEW user_details AS
    SELECT 
        user_id,
        CAST(user_data:name AS STRING) as name,
        CAST(user_data:email AS STRING) as email,
        CAST(user_data:age AS INT) as age,
        CAST(user_data:address.city AS STRING) as city,
        CAST(user_data:address.country AS STRING) as country,
        CAST(user_data:preferences.newsletter AS BOOLEAN) as newsletter_opt_in,
        CAST(user_data:created_at AS TIMESTAMP) as created_date,
        SIZE(CAST(user_data:profile.interests AS ARRAY<STRING>)) as interest_count
    FROM users_variant
""")

print("✅ Created structured view from VARIANT data")

spark.sql("SELECT * FROM user_details LIMIT 5").show()


=== Pivoting VARIANT Data ===



+--------------------+----+-----+-------+----------+
|             country|free|basic|premium|enterprise|
+--------------------+----+-----+-------+----------+
|         Afghanistan|NULL|    1|   NULL|      NULL|
|             Albania|   2| NULL|   NULL|      NULL|
|             Algeria|NULL|    3|      1|         1|
|      American Samoa|NULL| NULL|   NULL|         2|
|             Andorra|NULL| NULL|   NULL|      NULL|
|              Angola|NULL|    1|   NULL|      NULL|
|            Anguilla|NULL| NULL|   NULL|         1|
|Antarctica (the t...|NULL|    1|   NULL|      NULL|
| Antigua and Barbuda|   1|    2|   NULL|         3|
|           Argentina|   1|    3|      1|      NULL|
+--------------------+----+-----+-------+----------+


=== Creating Structured View from VARIANT ===

✅ Created structured view from VARIANT data


+--------------------+-----------------+--------------------+---+-------------------+----------+-----------------+--------------------+--------------+
|             user_id|             name|               email|age|               city|   country|newsletter_opt_in|        created_date|interest_count|
+--------------------+-----------------+--------------------+---+-------------------+----------+-----------------+--------------------+--------------+
|60cabef2-6c60-43d...|Melanie Gutierrez|taylorann@example...| 79|          Butlerton|     Qatar|            false|2023-04-26 15:38:...|             4|
|7e8614cf-da0b-458...|     Kelsey Welch|stephenrice@examp...| 33|         Port Diana| Greenland|             true|2020-11-26 23:12:...|             2|
|bc0268cb-3954-4f4...|   William Butler|jamescarter@examp...| 38|     Port Haroldton|Guadeloupe|             true|2024-06-26 07:18:...|             4|
|5fd673b7-34cd-413...|       Kayla Hall|garciaterri@examp...| 23| South Stephenville|     Beni

## 12. Cleanup Script

This section provides a comprehensive cleanup script to remove all resources created during this exploration.


In [24]:
# Cleanup Script - Removes all resources created during VARIANT exploration

def cleanup():
    """Simple cleanup function to remove all created resources"""
    print("🧹 Cleaning up VARIANT exploration resources...")
    
    # Drop tables
    tables = ['users_variant', 'users_extracted', 'variant_optimized']
    for table in tables:
        spark.sql(f"DROP TABLE IF EXISTS {table}")
        print(f"✅ Dropped table: {table}")
    
    # Drop views
    spark.sql("DROP VIEW IF EXISTS user_details")
    print("✅ Dropped view: user_details")
    
    # Clean volume files
    users_folder = f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/users"
    try:
        dbutils.fs.rm(users_folder, recurse=True)
        print("✅ Cleaned volume files")
    except:
        print("⚠️  Volume files may not exist")
    
    # Reset to default catalog/schema
    spark.sql("USE CATALOG main")
    spark.sql("USE SCHEMA default")
    print("✅ Reset to main catalog, default schema")
    
    print("\n🎉 Cleanup complete!")

# Run cleanup
cleanup()


🧹 Cleaning up VARIANT exploration resources...
✅ Dropped table: users_variant
✅ Dropped table: users_extracted
✅ Dropped table: variant_optimized
✅ Dropped view: user_details
✅ Cleaned volume files
✅ Reset to main catalog, default schema

🎉 Cleanup complete!


In [25]:
# Optional: Individual cleanup functions

# def drop_tables_only():
#     """Drop only the tables"""
#     tables = ['users_variant', 'users_extracted', 'variant_optimized']
#     for table in tables:
#         spark.sql(f"DROP TABLE IF EXISTS {table}")
#         print(f"✅ Dropped {table}")

# def clean_volume_only():
#     """Clean only the volume files"""
#     dbutils.fs.rm(f"/Volumes/{CATALOG}/{SCHEMA}/{VOLUME}/users", recurse=True)
#     print("✅ Cleaned volume files")

# Usage:
# cleanup()           # Full cleanup (already run above)
# drop_tables_only()  # Drop only tables
# clean_volume_only() # Clean only volume files
